In [ ]:
import numpy as np
from deap import base, creator, tools
import random

# 定义选择的数量
N = 10000

# 生成拉丁方
def generate_latin_square(n):
    latin_square = np.zeros((n, n), dtype=int)
    for i in range(n):
        latin_square[i] = (np.arange(n) + i) % n
    return latin_square

# 定义适应度函数
def fitness_function(ind):
    # 提取选择的索引
    i_ind = ind[:N]
    j_ind = ind[N:2*N]
    k_ind = ind[2*N:3*N]
    
    # 计算频数
    freq_i = np.bincount(i_ind, minlength=100)
    freq_j = np.bincount(j_ind, minlength=100)
    freq_k = np.bincount(k_ind, minlength=100)

    # 合并频数
    combined_freq = np.concatenate((freq_i, freq_j, freq_k))
    
    penality = 0
    # if(np.min(combined_freq) < 100):
    #     penality += (100 - np.min(combined_freq))
    # # 返回最小频数（作为适应度）
    # return np.min(combined_freq) - penality,

    
    penality += np.var(combined_freq)
    return np.min(combined_freq) - np.var(combined_freq),

    # return np.min(combined_freq),

# 遗传算法参数设置
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # 最大化
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# 使用拉丁方初始化个体
latin_square_i = generate_latin_square(100)  # 生成100x100的拉丁方用于i
latin_square_jk = generate_latin_square(100)   # 生成50x50的拉丁方用于j和k

toolbox.register("i_indices", random.choices, (lambda: latin_square_i[random.randint(0, 99)].tolist())(), k=N)  # i 的选择
toolbox.register("j_indices", random.choices, (lambda: latin_square_i[random.randint(0, 99)].tolist())(), k=N)  # j 的选择
toolbox.register("k_indices", random.choices, (lambda: latin_square_i[random.randint(0, 99)].tolist())(), k=N)  # k 的选择

toolbox.register("individual", tools.initIterate, creator.Individual, 
                 lambda: toolbox.i_indices() + toolbox.j_indices() + toolbox.k_indices())


toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# 遗传算法执行
population = toolbox.population(n=5000)  # 种群大小
ngen = 200  # 最大迭代次数
count = 0

for gen in range(ngen):
    # 选择
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # 交叉和变异
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:  # 交叉概率
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.3:  # 变异概率
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # 评估新一代个体
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # 替换种群
    population[:] = offspring

    # 输出最佳解
    fits = [ind.fitness.values[0] for ind in population]
    best_idx = np.argmax(fits)
    best_ind = population[best_idx]

    # 计算最佳解的最小频数
    best_freq_i = np.bincount(best_ind[:N], minlength=100)
    best_freq_j = np.bincount(best_ind[N:2*N], minlength=100)
    best_freq_k = np.bincount(best_ind[2*N:3*N], minlength=100)
    combined_best_freq = np.concatenate((best_freq_i, best_freq_j, best_freq_k))
    min_freq = np.min(combined_best_freq)

    count += 1
    print(f"迭代 {count} 次; 最佳最小频数:, {min_freq}")

# 计算最佳解的最小频数
best_freq_i = np.bincount(best_ind[:N], minlength=100)
best_freq_j = np.bincount(best_ind[N:2*N], minlength=100)
best_freq_k = np.bincount(best_ind[2*N:3*N], minlength=100)
combined_best_freq = np.concatenate((best_freq_i, best_freq_j, best_freq_k))
min_freq = np.min(combined_best_freq)

print(f"迭代 {ngen} 次: ")
print("最佳最小频数:", min_freq, combined_best_freq)
print(len(best_freq_i),len(best_freq_j),len(best_freq_k))

print("best_ind",best_ind)
print("best_ind-i", best_ind[:N])
print("best_ind-j", best_ind[N:2*N])
print("best_ind-k", best_ind[2*N:3*N])

迭代 1 次; 最佳最小频数:, 78
迭代 2 次; 最佳最小频数:, 78
迭代 2 次: 
最佳最小频数: 78 [ 88 101 106  99 101  80  96  91 111 101  95 101  95  96  97 106  95  98
  97 101 114  96 106 119  93 105 100 109  89 106 102  95  95 109  98  94
 110  95 105 101  96 111  92 102  99 100 102  93 114 100  97  92  87 105
 104 114  96 103 107 111  98  95 100 100 102 108 102  94 117 109  99  90
  93  89 105  87  93 100  99 106 106  85  92  95  93  89  88 119 101 100
 117 103 111  90  98  95 109  95 110  97 105 106 102  93  89  97  97  87
  84  95  92 104 109 114  95  92 107 113 100  95 115  89 111 115 108  98
  99 102 113 102  87 107  95 101  92  97 100 107  90 100 102 102  92 111
  98 100  95  94  80 104 111  97  85  94 104  97  90 101 111  99 116 101
 104 106  89  87 111 103  85  93  78 113 108  94 122 108 101 106 108 117
  97  99  97 105 113 107  93 104  84  93  94 101  96 109 107  87 103  93
  96 101  98 101 107 106  96 102  92 105  94  97 103  85 103 106  98 106
  96  91  93  90 109 102 115  92  84  84  99 100 104  99  98 100

In [22]:
len(combined_best_freq)

300

In [21]:
np.sum(combined_best_freq)

30000

In [42]:
def generate_latin_square(n):
    latin_square = np.zeros((n, n), dtype=int)
    for i in range(n):
        latin_square[i] = (np.arange(n) + i) % n
    return latin_square
latin_square_i = generate_latin_square(100) 

# latin_square_i[random.randint(0, 99)].tolist()
arr = lambda: latin_square_i[random.randint(0, 99)].tolist()
print(arr())

[77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]


In [50]:
random.choices(range(100),k=5)

[0, 90, 77, 64, 51]

In [47]:
a = np.array([0, 1090, 77, 64, 51])
b = sum(100 - a[a < 100])
b

208